In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'HTBot.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from Database.tasks import check_candles_and_open

check_candles_and_open()

Saved/updated 100 rows to candles table for DOGEUSDT (interval: 3600000ms).
{"code":"00000","msg":"success","requestTime":1757099465960,"data":{"symbol":"DOGEUSDT_UMCBL","markPrice":"0.216172","timestamp":"1757099465962"}}
{"code":"00000","msg":"success","requestTime":1757099466924,"data":{"clientOid":"1347938350765793280","orderId":"1347938350690295809"}}
{"code":"00000","msg":"success","requestTime":1757099467931,"data":{"symbol":"DOGEUSDT_UMCBL","markPrice":"0.21614","timestamp":"1757099467933"}}
{"code":"00000","msg":"success","requestTime":1757099468981,"data":{"clientOid":"1347938359297159168","orderId":"1347938359297159169"}}
{"code":"00000","msg":"success","requestTime":1757099470164,"data":{"clientOid":"1347938364196106240","orderId":"1347938364196106241"}}


In [4]:
trigger_price = 0.2227726700000000019590284950
round(trigger_price, 10)
u 1757082520498
c 1757082518970

0.22277267

In [4]:
from Database.tasks import check_position

check_position()

{"code":"43020","msg":"Stop profit and stop loss order does not exist","requestTime":1757099509691,"data":null}
response_code is 43020


In [8]:
from datetime import datetime, timedelta
months_ago_timestamp = (datetime.now() - timedelta(days=90)).timestamp() * 1000
print(months_ago_timestamp)

1749301974555.333


In [4]:
from Database.models import Candle
from ExchangeAPI.APICallManager import Interval
candles = Candle.objects.filter(symbol__symbol="DOGEUSDT",
                                interval=Interval.HOUR_1.value[1])

In [17]:
len(candles)

29904

In [18]:
from datetime import datetime
datetime.fromtimestamp(candles.first().open_time/1000)

datetime.datetime(2022, 4, 3, 15, 0)

In [19]:
from datetime import datetime
datetime.fromtimestamp(candles.last().open_time/1000)

datetime.datetime(2025, 8, 31, 14, 0)

In [15]:
from ExchangeAPI.APICallManager import CandleAgent
# from ExchangeAPI.APICallManager import Interval
agent = CandleAgent(symbol="DOGEUSDT", interval=Interval.HOUR_1)
# for i in range(15):
#     old_candles = agent.fetch_past_candles()
#     agent.save_to_db(candles=old_candles)
candles = agent.fetch_future_candles()
agent.save_to_db(candles=candles)
agent.check_candles_consistency()

Saved/updated 100 rows to candles table for DOGEUSDT (interval: 3600000ms).
OKAY


In [5]:
def will_success(tp_percentage, sl_percentage, sign_candle: Candle, position_type: str):
    future_candles = Candle.objects.filter(
        symbol=sign_candle.symbol,
        interval=sign_candle.interval,
        open_time__gt=sign_candle.open_time
    )
    price = sign_candle.close
    if position_type == "long":
        tp_price = price * (1 + tp_percentage / 100)
        sl_price = price * (1 - sl_percentage / 100)
        for future_candle in future_candles:
            if future_candle.high >= tp_price:
                return True, future_candle.open_time
            elif future_candle.low <= sl_price:
                return False, future_candle.open_time
        return False, datetime.now().timestamp() * 1000
    else:
        sl_price = price * (1 + sl_percentage / 100)
        tp_price = price * (1 - tp_percentage / 100)
        for future_candle in future_candles:
            if future_candle.high >= sl_price:
                return False, future_candle.open_time
            elif future_candle.low <= tp_price:
                return True, future_candle.open_time
        return False, datetime.now().timestamp() * 1000

In [6]:
from django.db.models import QuerySet
from Database.models import Candle

def strategy(candles: QuerySet[Candle], tp_percentage: int, sl_percentage: int):
    total_counts = 0
    success_counts = 0
    last_time = candles.first().open_time
    for two_previous_candle, previous_candle, candle in zip(candles, candles[1:], candles[2:]):

        if candle.open_time <= last_time:
            continue

        if two_previous_candle.is_green() and previous_candle.is_green() and candle.is_green():

            total_counts += 1
            successful, last_time = will_success(
                tp_percentage=tp_percentage,
                sl_percentage=sl_percentage,
                sign_candle=candle,
                position_type="long",
            )
            if successful:
                success_counts += 1
                # print("SUCCESS LONG")
                # print(datetime.fromtimestamp(candle.open_time / 1000))
                # print("**********************\n")
            else:
                pass
                # print("FAIL LONG")
                # print(datetime.fromtimestamp(candle.open_time / 1000))
                # print("**********************\n")
        elif (not two_previous_candle.is_green() and
              not previous_candle.is_green() and
              not candle.is_green()):
            total_counts += 1
            successful, last_time = will_success(
                tp_percentage=tp_percentage,
                sl_percentage=sl_percentage,
                sign_candle=candle,
                position_type="short",
            )
            if successful:
                # print("SUCCESS SHORT")
                # print(datetime.fromtimestamp(candle.open_time / 1000))
                # print("**********************\n")
                success_counts += 1
            else:
                # print("FAIL SHORT")
                # print(datetime.fromtimestamp(candle.open_time / 1000))
                # print("**********************\n")
                pass

    return total_counts, success_counts

In [7]:
from datetime import datetime, timedelta
first_parameter = 0
last_parameter = 30
tp_percentage = 8
sl_percentage = 1
total_point = 0
i = 1

while last_parameter <= 36 * 30:
    end_time = (datetime.now() - timedelta(days=first_parameter)).timestamp() * 1000
    start_time = (datetime.now() - timedelta(days=last_parameter)).timestamp() * 1000
    print(datetime.fromtimestamp(start_time/1000))
    candles = Candle.objects.filter(symbol__symbol="DOGEUSDT",
                                open_time__gte=start_time,
                                open_time__lte=end_time,
                                interval=Interval.HOUR_1.value[1])
    total, success = strategy(
        candles=candles,
        tp_percentage=tp_percentage,
        sl_percentage=sl_percentage,
    )

    fail = total - success
    point = (success * tp_percentage) - (fail * sl_percentage)
    total_point += point
    first_parameter = last_parameter
    last_parameter += 30
    print(f'Month {i}: {(total, success)} -> {point}')
    i += 1

print(total_point)

2025-08-06 09:07:11.741287
Month 1: (34, 6) -> 20
2025-07-07 09:07:12.190500
Month 2: (43, 7) -> 20
2025-06-07 09:07:13.298051
Month 3: (31, 6) -> 23
2025-05-08 09:07:14.212170
Month 4: (44, 7) -> 19
2025-04-08 09:07:16.007152
Month 5: (34, 6) -> 20
2025-03-09 09:07:20.000565
Month 6: (43, 8) -> 29
2025-02-07 09:07:25.581016
Month 7: (47, 6) -> 7
2025-01-08 09:07:32.036379
Month 8: (61, 8) -> 11
2024-12-09 09:07:44.894241


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f1837c682c0>>
Traceback (most recent call last):
  File "/home/benyamin/Projects/HTBot/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Month 9: (42, 4) -> -6
2024-11-09 09:07:55.851819
Month 10: (63, 8) -> 9
2024-10-10 09:08:03.641569
Month 11: (37, 8) -> 35
2024-09-10 09:08:08.249847
Month 12: (29, 4) -> 7
2024-08-11 09:08:12.880438
Month 13: (35, 3) -> -8
2024-07-12 09:08:21.270153
Month 14: (41, 7) -> 22
2024-06-12 09:08:34.514181
Month 15: (32, 5) -> 13
2024-05-13 09:08:41.337096
Month 16: (36, 4) -> 0
2024-04-13 09:08:50.340475
Month 17: (55, 6) -> -1
2024-03-14 09:09:03.733307
Month 18: (68, 8) -> 4
2024-02-13 09:09:19.622387
Month 19: (57, 9) -> 24
2024-01-14 09:09:34.210607
Month 20: (9, 2) -> 9
2023-12-15 09:09:36.833072
Month 21: (35, 1) -> -26
2023-11-15 09:09:46.926796
Month 22: (44, 7) -> 19
2023-10-16 09:09:59.700798
Month 23: (33, 4) -> 3
2023-09-16 09:10:07.929635
Month 24: (8, 1) -> 1
2023-08-17 09:10:09.546670
Month 25: (17, 1) -> -8
2023-07-18 09:10:12.770721
Month 26: (31, 3) -> -4
2023-06-18 09:10:19.344340
Month 27: (35, 2) -> -17
2023-05-19 09:10:27.385730
Month 28: (13, 4) -> 23
2023-04-19 09:1